In [0]:
%sql
CREATE DATABASE IF NOT EXISTS presentation
LOCATION "/mnt/presentation";
use presentation;

In [0]:
%run "../commons/configuration"

In [0]:
races_df = spark.read.parquet(f"{processed_mount_path}/races")\
    .withColumnRenamed("name","race_name")\
    .withColumnRenamed("race_timestamp","race_date")

In [0]:
circuits_df = spark.read.parquet(f"{processed_mount_path}/circuits")\
    .withColumnRenamed("location","circuit_location")

In [0]:
results_df = spark.read.parquet(f"{processed_mount_path}/results")\
    .withColumnRenamed("time","race_time")

In [0]:
constructors_df = spark.read.parquet(f"{processed_mount_path}/constructors")\
    .withColumnRenamed("name","team")
    

In [0]:
drivers_df = spark.read.parquet(f"{processed_mount_path}/drivers")\
    .withColumnRenamed("name","driver_name")\
    .withColumnRenamed("nationality","driver_nationality")\
    .withColumnRenamed("number","driver_number")

In [0]:
races_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner")\
    .select(races_df.race_id, races_df.race_year, races_df.race_name, races_df.race_date, circuits_df.circuit_location)

In [0]:
race_results_df = results_df.join(races_circuits_df, results_df.race_id == races_circuits_df.race_id)\
                    .join(drivers_df, results_df.driver_id == drivers_df.driver_id)\
                    .join(constructors_df, results_df.constructor_id == constructors_df.constructor_id)

In [0]:
from pyspark.sql.functions import current_timestamp

final_df = race_results_df.select("race_year","race_name","race_date","circuit_location","driver_name","driver_number",
                                  "driver_nationality", "team","grid","fastest_lap","race_time","points","position")\
                            .withColumn("created_date", current_timestamp())

In [0]:
#final_df.write.parquet(f"{presentation_mount_path}/race_results", mode="overwrite")
final_df.write.mode("overwrite").format("parquet").saveAsTable("presentation.race_results")